# ゼロから作る Deep Learning 3 フレームワーク編

## ステップ1 箱としての変数
## ステップ6 手作業によるバックプロパゲーション

In [51]:
class Variable:
  """ 変数クラス
  
  使用する変数を格納するためのクラスである。
  
  Attributes:
    data: 対象の変数
    grad: data を微分した値
  """
  def __init__(self, data):
    """ 初期化
    
    Args:
      data: 変数
    """
    self.data = data
    self.grad = None

In [40]:
import numpy as np

data = np.array(1.0)
x =Variable(data)
print(x.data)

1.0


## ステップ2 Function クラスの実装
## ステップ6 手作業によるバックプロパゲーション

In [52]:
class Function:
  """ 関数クラス
  
  関数の基底クラスとして、全ての関数に共通する機能を実装する。
  具体的な関数は Function クラスを継承したクラスで実装する。  
  """
  def __call__(self, input):
    """ 関数の実行
    
    継承クラスにて実装した関数を実行する。
    
    Args:
      input(Variable): 関数の入力値
    
    Returns:
      Variable: 関数の実行結果    
    """ 
    x = input.data # データを取り出す。
    y = self.forward(x) # 具体的な計算は forward メソッドで行う。
    output = Variable(y) # Variable として返却する。
    self.input = input # 入力された変数を保持しておく。
    return output
  
  def forward(self, x):
    """ 関数のメソッド
    
    継承したクラスで実装すること。
    
    Args:
      x(Variable): 関数の入力値
    
    Raises:
      NotImplementedError: forward メソッドを実装せずに実行した場合に発生する。    
    """
    raise NotImplementedError()
  
  def backward(self, gy):
    """ 逆伝播(バックプロパゲーション)用のメソッド
    
    継承したクラスで実装すること。

    Args:
      gy(Variable): 出力側から伝わる微分の値
          
    Raises:
      NotImplementedError: forward メソッドを実装せずに実行した場合に発生する。    
    """
    raise NotImplementedError()

In [57]:
class Square(Function):
  """ 二乗計算クラス
  
  入力値の二乗を計算する。
  """
  def forward(self, x):
    """ 二乗計算
  
    入力値の二乗を計算する。
    
    Args:
      x(Variable): 入力値
    
    Returns:
      Variable: 二乗した結果
    """
    y = x ** 2
    return y

  def backward(self, gy):
    """ 二乗計算の逆伝播メソッド
    
    二乗計算の逆伝播として、出力側から渡される微分 (gy) と y=x^2 の微分を掛ける。
    
    Args:
      x(Variable): 出力側から伝わる微分の値
    
    Returns:
      Variable: 二乗計算のバックプロパゲーション結果
    """
    x = self.input.data
    gx = 2 * x * gy
    return gx

In [47]:
x = Variable(np.array(3.5))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
12.25


## ステップ3 関数の連結
## ステップ6 手作業によるバックプロパゲーション

In [58]:
class Exp(Function):
  """ EXP 関数
  
  e を底とする数値のべき乗を計算する。  
  y = e^x の実装 (e はネイピア数 2.718...)
  """  
  def forward(self, x):
    """ e のべき乗を計算する。
  
    自然対数 e を入力値でべき乗する。
    
    Args:
      x(Variable): 入力値
    
    Returns:
      Variable: e を x でべき乗した結果
    """
    y = np.exp(x)
    return y
  
  def backward(self, gy):
    """ e のべき乗の逆伝播メソッド
    
    e のべき乗の逆伝播として、出力側から渡される微分 (gy) と e^x の微分である e^x を掛ける。
    
    Args:
      x(Variable): 出力側から伝わる微分の値
    
    Returns:
      Variable: e のべき乗のバックプロパゲーション結果
    """
    x = self.input.data
    gx = np.exp(x) * gy
    return gx

In [31]:
A = Square()
B = Exp()
C = Square()

x =Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


## ステップ4 数値微分

In [ ]:
def numerical_diff(f, x, eps=1e-4):
  """ 数値微分
  中心差分近似で数値微分を求める。
  {f(x+h) - f(x-h)} / 2h
  
  Args:
    f (Function): 微分の対象となる関数
    x (Variable): 微分を求める変数
    eps (float, optional): 微小な数値 (初期値は 1e-4)

  Returns:
    float: 数値微分の結果
  """
  x0 = Variable(x.data - eps)
  x1 = Variable(x.data + eps)
  y0 = f(x0)
  y1 = f(x1)

  return (y1.data - y0.data) / (2 * eps)

In [37]:
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(type(dy))

<class 'numpy.float64'>


In [50]:
# 合成関数の微分
# y = (e^(x^2))^2
def f(x):
  A = Square()
  B = Exp()
  C = Square()
  return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694


## ステップ6 手作業によるバックプロパゲーション

In [62]:
# 順伝播
# x(0.5) => A(x^2) => B(e^a) => C(b^2) => y
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 逆伝播
# 1 => y.grad => C.backward => b.grad => B.backward => a.grad => A.backward => x.grad
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)

print(x.grad)

3.297442541400256
